In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm import tqdm
import re
import numba as nb
import matplotlib.pyplot as plt
import numba as nb
from transformers import BertTokenizer, AutoTokenizer, AutoModelForSequenceClassification

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import collections
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanamazondata/__notebook_source__.ipynb
/kaggle/input/cleanamazondata/clean.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/sample_submission.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/train.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv


In [2]:
train_vec = pd.read_csv('../input/cleanamazondata/clean.csv')#.sample(250000)
#train_vec = pd.read_csv('../input/amazontrainclean/out.csv')

In [3]:
train_vec = train_vec[484668*3:484668*4]

In [4]:
train_vec.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
y = train_vec['BROWSE_NODE_ID'].to_numpy()

In [6]:
with open('y_p4.npy', 'wb') as f:
    np.save(f,y)

In [7]:
del y

In [8]:
pip install sentence_transformers

     |████████████████████████████████| 85 kB 862 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=7d908934e4bc68a1b1c6d67ca70512231c6d1432e4568966c1b1a57ccf7b1415
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [9]:
from sentence_transformers import SentenceTransformer
model_1 = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_roberta-large')

inp=[]
#t= train_vec['TITLE'].tolist()
#d = train_vec['DESCRIPTION'].tolist()
#b = train_vec['BULLET_POINTS'].tolist()
brand = train_vec['BRAND'].tolist()
#br = {j:i for i,j in enumerate(brand)}

@nb.jit(forceobj=True,parallel=True)
def loop():
    for i in tqdm(range(len(train_vec))):
        sentence_a = train_vec['TITLE'].iloc[i]
        sentence_b = train_vec['DESCRIPTION'].iloc[i]
        sentence_c = train_vec['BULLET_POINTS'].iloc[i]
      
            
        if sentence_b!='none':
            s = sentence_b.replace(r"[^A-Za-z0-9]+", "").strip() +". "+sentence_c.replace(r"[^A-Za-z0-9]+", "").strip() +". "+ brand[i]
            s = s.replace('[','').replace(']','')
            #encoding = model.encode(s).tolist()
        else:
            s = sentence_a.replace(r"[^A-Za-z0-9]+", "").strip() + sentence_c.replace(r"[^A-Za-z0-9]+", "").strip() +". "+ brand[i]
            s = s.replace('[','').replace(']','')
            #encoding = model.encode(s).tolist()

        inp.append(s)
    #tkn.append(encoding['token_type_ids'])
    #msk.append(encoding['attention_mask'])
    #data.append(encoding['input_ids']*encoding['token_type_ids']*encoding['attention_mask']+train_vec['BRAND'].tolist()[i])

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [10]:
loop()

100%|██████████| 484668/484668 [00:20<00:00, 23457.77it/s]


In [11]:
del train_vec,brand

In [12]:
inp = model_1.encode(inp)

Batches:   0%|          | 0/15146 [00:00<?, ?it/s]

In [13]:
# del inp

In [14]:
with open('encoding_p4.npy', 'wb') as f:
    np.save(f,inp)